<a href="https://colab.research.google.com/github/LouisYLWang/SCF-2-SN/blob/main/SCF_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To check R version
R.version.string

[1] "R version 4.2.0 (2022-04-22)"

#**Installing packages and libraries**

In [ ]:
#install packages and load libraries
library(dplyr)
library(readxl)
install.packages("xlsx", INSTALL_opts=c("--no-multiarch")) #install package xlsx this way to be compatible w/ 64-bit Java only
library(xlsx)
# install.packages("tidyr")
library(tidyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rJava’, ‘xlsxjars’




**Requirements:**

**1.   “SCF 2021.3” sheet**


*   SCF Domain: 32 total domains in string format, full list sees “SCF Domains & Principles” sheet
*   SCF Control: name of each control in string format
*   SCF #: ID of each control in string format
*   Control Description: Long string, when importing, should be concatenated with “Method To Comply With SCF Controls” and “SCF Control Question”
*   Method To Comply With SCF Controls: Long string
*   SCF Control Question: Long string
*   Function Grouping: 5 total function groups in string format 
*   Regulation and chapter ID: columns after columns R to IQ provide information about how SCF controls map to each framework and regulation. Value in each cell stands for the related chapters of control in each row. 

**2.   “Authoritative Sources” sheet:**
*   Version: string
*   URL: string




---




**Cleaning Tasks**

1. Drop unnessary columns (for both sheets) and check column names: Done
2. Check the column datatypes to match above requirements (in R, chr is equivalent to string format) -- use is.character and as.character: Done
3. Concatenate: "Secure Controls Framework (SCF) Control Description", “Method To Comply With SCF Controls”, and “SCF Control Question” columns: Done
4. Concatenate SCF Number (#) and SCF Control columns for the column Name in Control Objective target table: Done
5. Remove duplicates from Authoritative Source and Version columns in Authoritative Sources sheet: Done
6. Map multiple chapter IDs (Citations) correctly to the column Name in Control Objective target table (or SCF # and SCF Control columns in source data)
7. Export both sheets into CSV/JDBC/XML (choose CSV)
8. Relationship between target tables: Each Authority Document contains one or multiple Citations, each of which in turn contains a Control Objective that defines a risk category: Done
9. For the citation table, split multiple values in the column in multiple rows




#**Cleaning SCF 2021.3 sheet**

In [ ]:
# Importing scf sheet (2021 Version)
# scf = read_excel("/content/Secure Controls Framework (SCF) - 2021.3.xlsx")

#Importing scf sheet (Anh's version)
scf = read_excel("/content/SCF-2021_3 raw data.xlsx")

# Checking first few rows of scf sheet
head(scf, n=1)

# Checking last few rows of scf sheet
tail(scf, n=1)

# Check dimensions
dim(scf)

SCF Domain,SCF Control,SCF #,Secure Controls Framework (SCF) Control Description,Methods To Comply With SCF Controls,SCF Control Question,Relative Control Weighting (1-10),Function Grouping,SCRM Tier 1 Strategic,SCRM Tier 2 Operational,⋯,Threat MT-4,Threat MT-5,Threat MT-6,Threat MT-7,Threat MT-8,Threat MT-9,Threat MT-10,SCF Errata 2021.1,SCF Errata 2021.2,SCF Errata 2021.3
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Security & Privacy Governance,Security & Privacy Governance Program,GOV-01,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls.,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,10,Identify,X,X,⋯,NA,NA,NA,MT-7,MT-8,MT-9,NA,ISO 27001 6.1.1,NA,- ISO 27001 added 4.3 & 4.4


SCF Domain,SCF Control,SCF #,Secure Controls Framework (SCF) Control Description,Methods To Comply With SCF Controls,SCF Control Question,Relative Control Weighting (1-10),Function Grouping,SCRM Tier 1 Strategic,SCRM Tier 2 Operational,⋯,Threat MT-4,Threat MT-5,Threat MT-6,Threat MT-7,Threat MT-8,Threat MT-9,Threat MT-10,SCF Errata 2021.1,SCF Errata 2021.2,SCF Errata 2021.3
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Web Security,Web Browser Security,WEB-12,"Mechanisms exist to ensure web applications implement Content-Security-Policy, HSTS and X-Frame-Options response headers to protect both the web application and its users.",NA,"Does the organization ensure web applications implement Content-Security-Policy, HSTS and X-Frame-Options response headers to protect both the web application and its users?",9,Protect,NA,X,⋯,NA,NA,NA,MT-7,MT-8,MT-9,NA,New addition,NA,NA


[1] 1006  251

In [ ]:
# Check missing values
colSums(is.na(scf))

# Drop unnecesary columns from scf sheet
drop <- c("Relative Control\r\nWeighting\r\n(1-10)", "SCRM\r\nTier 1\r\nStrategic", "SCRM\r\nTier 2\r\nOperational", "SCRM\r\nTier 3\r\nTactical",
          "SP-CMM 0\r\nNot Performed", "SP-CMM 1\r\nPerformed Informally", "SP-CMM 2\r\nPlanned & Tracked", "SP-CMM 3\r\nWell Defined", 
          "SP-CMM 4\r\nQuantitatively Controlled", "SP-CMM 5\r\nContinuously Improving")
scf = scf[,!(names(scf) %in% drop)]

# Check df dimensions after dropping
dim(scf)

# Check datatypes of all columns again to make sure they match specs doc
glimpse(scf)

SCF Domain 
                                                     0 
                                           SCF Control 
                                                     0 
                                                 SCF # 
                                                     0 
Secure Controls Framework (SCF)\r\nControl Description 
                                                     0 
                   Methods To Comply With SCF Controls 
                                                   602 
                                  SCF Control Question 
                                                     0 
               Relative Control\r\nWeighting\r\n(1-10) 
                                                     0 
                                     Function Grouping 
                                                     0 
                           SCRM\r\nTier 1\r\nStrategic 
                                                   847 
                         SCRM\r\nTier 2\r\nOperational 
                                                   213 
                            SCRM\r\nTier 3\r\nTactical 
                                                   314 
                             SP-CMM 0\r\nNot Performed 
                                                     0 
                      SP-CMM 1\r\nPerformed Informally 
                                                     0 
                         SP-CMM 2\r\nPlanned & Tracked 
                                                     0 
                              SP-CMM 3\r\nWell Defined 
                                                     0 
                 SP-CMM 4\r\nQuantitatively Controlled 
                                                     0 
                    SP-CMM 5\r\nContinuously Improving 
                                                     0 
                          AICPA\r\nTSC 2017\r\n(SOC 2) 
                                                   723 
                                    CIS\r\nCSC\r\nv7.1 
                                                   817 
                                    CIS\r\nCSC\r\nv8.0 
                                                   799 
                                         COBIT\r\n2019 
                                                   857 
                                         COSO\r\nv2017 
                                                   981 
                                  CSA\r\nCCM\r\nv3.0.1 
                                                   892 
                                      CSA\r\nCCM\r\nv4 
                                                   673 
                                  CSA\r\nIoT SCF\r\nv2 
                                                   745 
                                         ENISA\r\nv2.0 
                                                   945 
                                                  GAPP 
                                                   957 
                                         IEC 62443-4-2 
                                                   880 
                                 ISO\r\n22301\r\nv2019 
                                                   975 
                                 ISO\r\n27001\r\nv2013 
                                                   961 
                                 ISO\r\n27002\r\nv2013 
                                                   819 
                                ISO \r\n27018\r\nv2014 
                                                   989 
                                ISO\r\n27701 \r\nv2019 
                                                   812 
                                 ISO\r\n29100\r\nv2011 
                                                   973 
                                 ISO\r\n31000\r\nv2009 
                                                   988 
                                 ISO\r\n31010\r\nv2009 
                                                   977 
            

[1] 1006  241

Rows: 1,006
Columns: 241
$ `SCF Domain`                                             <chr> "Security & P…
$ `SCF Control`                                            <chr> "Security & P…
$ `SCF #`                                                  <chr> "GOV-01", "GO…
$ `Secure Controls Framework (SCF)\r\nControl Description` <chr> "Mechanisms e…
$ `Methods To Comply With SCF Controls`                    <chr> "- Steering c…
$ `SCF Control Question`                                   <chr> "Does the org…
$ `Function Grouping`                                      <chr> "Identify", "…
$ `AICPA\r\nTSC 2017\r\n(SOC 2)`                           <chr> "CC1.2", NA, …
$ `CIS\r\nCSC\r\nv7.1`                                     <chr> NA, NA, NA, N…
$ `CIS\r\nCSC\r\nv8.0`                                     <chr> NA, NA, NA, N…
$ `COBIT\r\n2019`                                          <chr> "EDM01.02\r\n…
$ `COSO\r\nv2017`                                          <chr> "Principle 2"…
$ `CSA\r\nCCM\r

#### **Concatenation Tasks**

In [ ]:
# Concatenate "Secure Controls Framework" column with “Methods To Comply With SCF Controls” and “SCF Control Question” columns into a new column
scf$"Control Objective Description" <- paste(scf$"Secure Controls Framework (SCF)\r\nControl Description", " ", scf$"Methods To Comply With SCF Controls", " ", 
scf$"SCF Control Question")

# Concatenate "SCF Control" and "SCF Number(#)" columns into a new column
scf$"Control Objective Name" <- paste(scf$"SCF Control", " ", scf$"SCF #")

In [ ]:
# Drop "Secure Controls Framework", "SCF Control" and "SCF #" columns to match columns specified in specs doc
drop <- c("Secure Controls Framework (SCF)\r\nControl Description", "SCF Control", "SCF #")
scf_cleaned = scf[,!(names(scf) %in% drop)] 

In [ ]:
# Move "Control Objective Description" column to before "Methods to Comply" column
scf_cleaned <- scf_cleaned %>% 
  relocate("Control Objective Description", .before = "Methods To Comply With SCF Controls")

# Move "Control Objective Name" column to before "Methods to Comply" column
scf_cleaned <- scf_cleaned %>% 
  relocate("Control Objective Name", .before = "SCF Domain")

In [ ]:
# Check first few rows again
head(scf_cleaned, 1)

Control Objective Name,SCF Domain,Control Objective Description,Methods To Comply With SCF Controls,SCF Control Question,Function Grouping,AICPA TSC 2017 (SOC 2),CIS CSC v7.1,CIS CSC v8.0,COBIT 2019,⋯,Threat MT-4,Threat MT-5,Threat MT-6,Threat MT-7,Threat MT-8,Threat MT-9,Threat MT-10,SCF Errata 2021.1,SCF Errata 2021.2,SCF Errata 2021.3
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,Identify,CC1.2,NA,NA,EDM01.02 APO01.09 APO04.01 APO13.01 APO13.02,⋯,NA,NA,NA,MT-7,MT-8,MT-9,NA,ISO 27001 6.1.1,NA,- ISO 27001 added 4.3 & 4.4


# **Transposing scf_cleaned Sheet**

In [ ]:
# Transpose columns G thru IF from wide to long format into 2 columns called "regulation" and
# "chapterID" to map onto field called "Name" in Authority Document target table and 
# field called "Name" in Citation target table
# see https://tidyr.tidyverse.org/reference/pivot_longer.html
scf_cleaned_transposed <- scf_cleaned %>%
  pivot_longer(
    cols = "AICPA\r\nTSC 2017\r\n(SOC 2)":"SCF Errata\r\n2021.3",
    names_to = "regulation",
    values_to = "chapterID"
  )

In [ ]:
# Check to see if transpose ran correctly for columns "regulation" and "chapterID"
select(scf_cleaned_transposed, regulation, chapterID, "Control Objective Name")

regulation,chapterID,Control Objective Name
<chr>,<chr>,<chr>
AICPA TSC 2017 (SOC 2),CC1.2,Security & Privacy Governance Program GOV-01
CIS CSC v7.1,NA,Security & Privacy Governance Program GOV-01
CIS CSC v8.0,NA,Security & Privacy Governance Program GOV-01
COBIT 2019,EDM01.02 APO01.09 APO04.01 APO13.01 APO13.02,Security & Privacy Governance Program GOV-01
COSO v2017,Principle 2,Security & Privacy Governance Program GOV-01
CSA CCM v3.0.1,GRM-04,Security & Privacy Governance Program GOV-01
CSA CCM v4,GRC-05 GRC-07,Security & Privacy Governance Program GOV-01
CSA IoT SCF v2,GVN-01 GVN-02,Security & Privacy Governance Program GOV-01
ENISA v2.0,NA,Security & Privacy Governance Program GOV-01


# **Extracting Regulations and Removing Duplicates** 

In [ ]:
# Extracting regulations from the transposed df
source_auth_doc = scf_cleaned_transposed['regulation']
dim(source_auth_doc) #check dimension

# Removing duplicate regulations
source_auth_doc = source_auth_doc[!duplicated(source_auth_doc$regulation), ]

# Checking the df
head(source_auth_doc, 5)
dim(source_auth_doc) 

[1] 235404      1

regulation
<chr>
AICPA TSC 2017 (SOC 2)
CIS CSC v7.1
CIS CSC v8.0
COBIT 2019
COSO v2017


[1] 234   1

#**Extracting Citations and Removing Duplicates**

In [ ]:
# Since some cells contain >1 chapter ID, separate such chapter IDs into separate rows
scf_cleaned_transposed_separate_chapterID = separate_rows(scf_cleaned_transposed,"chapterID", sep="\n")

In [ ]:
# Check to make sure chapter IDs separated properly into individual rows
select(scf_cleaned_transposed_separate_chapterID, regulation, chapterID)

regulation,chapterID
<chr>,<chr>
AICPA TSC 2017 (SOC 2),CC1.2
CIS CSC v7.1,NA
CIS CSC v8.0,NA
COBIT 2019,EDM01.02
COBIT 2019,APO01.09
COBIT 2019,APO04.01
COBIT 2019,APO13.01
COBIT 2019,APO13.02
COSO v2017,Principle 2


In [ ]:
#keep only relevant columns for Control Objectives to Citations relationship table in SN
source_citation_to_control_objective = scf_cleaned_transposed_separate_chapterID[, c("Control Objective Name", "SCF Domain", "Control Objective Description", 
"Methods To Comply With SCF Controls", "SCF Control Question", "chapterID", "regulation")]

In [ ]:
head(source_citation_to_control_objective, 5) #check the first few rows 
dim(source_citation_to_control_objective) #check the df dimension

Control Objective Name,SCF Domain,Control Objective Description,Methods To Comply With SCF Controls,SCF Control Question,chapterID,regulation
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,CC1.2,AICPA TSC 2017 (SOC 2)
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,NA,CIS CSC v7.1
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,NA,CIS CSC v8.0
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,EDM01.02,COBIT 2019
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,APO01.09,COBIT 2019


[1] 282862      7

In [ ]:
#Remove duplicate PAIRS of regulations and chapter IDs
source_citation = source_citation_to_control_objective[!duplicated(source_citation_to_control_objective[c("chapterID","regulation")]),]

#Alternative method for removing duplicate PAIRS of regulations and chapter IDs 
#distinct(source_citation_to_control_objective, chapterID, regulation, .keep_all= TRUE)

head(source_citation, 5) #check the first few rows of new df
dim(source_citation) #check the new df dimension

#test removing duplicates on regulation, chapter ID, and control objective name columns
#source_test = source_citation_to_control_objective[!duplicated(source_citation_to_control_objective[c("chapterID","Control Objective Name", "regulation")]),]
#head(source_test, 5) #check the first few rows of new df
#dim(source_test) #check the new df dimension

Control Objective Name,SCF Domain,Control Objective Description,Methods To Comply With SCF Controls,SCF Control Question,chapterID,regulation
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,CC1.2,AICPA TSC 2017 (SOC 2)
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,NA,CIS CSC v7.1
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,NA,CIS CSC v8.0
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,EDM01.02,COBIT 2019
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,APO01.09,COBIT 2019


[1] 17838     7

#**Exporting cleaned data**

In [ ]:
# Export cleaned scf sheet into new Excel file
write.xlsx(scf_cleaned, file = "/content/scf_auth_cleaned.xlsx", sheetName = "SCF 2021.3_cleaned", col.names = TRUE, row.names = TRUE, append = FALSE)

# Export cleaned scf sheet into new CSV file for Control Objectives table in SN
write.csv(scf_cleaned,"/content/scf_cleaned.csv", row.names = FALSE)

In [ ]:
# Export regulations into new CSV file for Authority Docs table in SN
write.csv(source_auth_doc,"/content/regulations.csv", row.names = FALSE)

In [ ]:
# Export citations into new CSV file for Citations table in SN
write.csv(source_citation,"/content/citations.csv", row.names = FALSE)

In [ ]:
# Export Control Objectives to Citations into new CSV file for relationship table between 
# Control Objectives and Citations in SN
write.csv(source_citation_to_control_objective,"/content/citations_to_control_objectives.csv", row.names = FALSE)

In [ ]:
# Export transposed data into new CSV file for reference
write.csv(scf_cleaned_transposed,"/content/scf_cleaned_transposed.csv", row.names = FALSE)

In [ ]:
# Export cleaned authoritative sheet into separate new CSV file
# write.csv(auth_cleaned,"/content/auth_cleaned.csv", row.names = FALSE)

#**Cleaning SCF NEW sheet**

In [ ]:
#Importing 2022 scf sheet
scf_new = read_excel("/content/SCF_current.xlsx")

# Checking first few rows of scf sheet
head(scf_new, n=1)

# Checking last few rows of scf sheet
tail(scf_new, n=1)

# Check dimensions
dim(scf_new)

# Check missing values
colSums(is.na(scf_new))

# Drop unnecesary columns from scf sheet
drop <- c("Relative Control\r\nWeighting\r\n(1-10)", "SCRM\r\nTier 1\r\nStrategic", "SCRM\r\nTier 2\r\nOperational", "SCRM\r\nTier 3\r\nTactical",
          "SP-CMM 0\r\nNot Performed", "SP-CMM 1\r\nPerformed Informally", "SP-CMM 2\r\nPlanned & Tracked", "SP-CMM 3\r\nWell Defined", 
          "SP-CMM 4\r\nQuantitatively Controlled", "SP-CMM 5\r\nContinuously Improving")
scf_new = scf_new[,!(names(scf_new) %in% drop)]

# Check df dimensions after dropping
dim(scf_new)

# Check datatypes of all columns again to make sure they match specs doc
glimpse(scf_new)

#### **Concatenation Tasks NEW**

In [ ]:
# Concatenate "Secure Controls Framework" column with “Methods To Comply With SCF Controls” and “SCF Control Question” columns into a new column
scf_new$"Control Objective Description" <- paste(scf_new$"Secure Controls Framework (SCF)\r\nControl Description", " ", scf_new$"Methods To Comply With SCF Controls", " ", 
scf_new$"SCF Control Question")

# Concatenate "SCF Control" and "SCF Number(#)" columns into a new column
scf_new$"Control Objective Name" <- paste(scf_new$"SCF Control", " ", scf_new$"SCF #")

# Drop "Secure Controls Framework", "SCF Control" and "SCF #" columns to match columns specified in specs doc
drop <- c("Secure Controls Framework (SCF)\r\nControl Description", "SCF Control", "SCF #")
scf_new_cleaned = scf_new[,!(names(scf_new) %in% drop)] 

# Move "Control Objective Description" column to before "Methods to Comply" column
scf_new_cleaned <- scf_new_cleaned %>% 
  relocate("Control Objective Description", .before = "Methods To Comply With SCF Controls")

# Move "Control Objective Name" column to before "Methods to Comply" column
scf_new_cleaned <- scf_new_cleaned %>% 
  relocate("Control Objective Name", .before = "SCF Domain")

# Check first few rows again
head(scf_new_cleaned, 1)

# **Transposing scf_NEW_cleaned Sheet**

In [ ]:
# Transpose columns G thru IF from wide to long format into 2 columns called "regulation" and
# "chapterID" to map onto field called "Name" in Authority Document target table and 
# field called "Name" in Citation target table
# see https://tidyr.tidyverse.org/reference/pivot_longer.html
scf_new_cleaned_transposed <- scf_new_cleaned %>%
  pivot_longer(
    cols = "AICPA\r\nTSC 2017\r\n(SOC 2)":"SCF Errata\r\n2022.1",
    names_to = "regulation",
    values_to = "chapterID"
  )

# Check to see if transpose ran correctly for columns "regulation" and "chapterID"
select(scf_new_cleaned_transposed, regulation, chapterID)  

# **Extracting Regulations and Removing Duplicates NEW** 

In [ ]:
# Extracting regulations from the transposed df
source_auth_doc_new = scf_new_cleaned_transposed['regulation']
dim(source_auth_doc_new) #check dimension

# Removing duplicate regulations
source_auth_doc_new = source_auth_doc_new[!duplicated(source_auth_doc_new$regulation), ]

# Checking the df
head(source_auth_doc_new, 5)
dim(source_auth_doc_new) 

regulation
<chr>
AICPA TSC 2017 (SOC 2)
CIS CSC v7.1
CIS CSC v8.0
COBIT 2019
COSO v2017


[1] 246   1

#**Extracting Citations and Removing Duplicates NEW**

In [ ]:
# Since some cells contain >1 chapter ID, separate such chapter IDs into separate rows
scf_new_cleaned_transposed_separate_chapterID = separate_rows(scf_new_cleaned_transposed,"chapterID", sep="\n")

In [ ]:
# Check to make sure chapter IDs separated properly into individual rows
select(scf_new_cleaned_transposed_separate_chapterID, regulation, chapterID)

regulation,chapterID
<chr>,<chr>
AICPA TSC 2017 (SOC 2),CC1.2
CIS CSC v7.1,NA
CIS CSC v8.0,NA
COBIT 2019,EDM01.02
COBIT 2019,APO01.09
COBIT 2019,APO04.01
COBIT 2019,APO13.01
COBIT 2019,APO13.02
COSO v2017,Principle 2


In [ ]:
#keep only relevant columns for Control Objectives to Citations relationship table in SN
source_citation_to_control_objective_new = scf_new_cleaned_transposed_separate_chapterID[, c("Control Objective Name", "SCF Domain", "Control Objective Description", 
"Methods To Comply With SCF Controls", "SCF Control Question", "chapterID", "regulation")]

In [ ]:
head(source_citation_to_control_objective_new, 5) #check the first few rows 
dim(source_citation_to_control_objective_new) #check the df dimension

Control Objective Name,SCF Domain,Control Objective Description,Methods To Comply With SCF Controls,SCF Control Question,chapterID,regulation
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,CC1.2,AICPA TSC 2017 (SOC 2)
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,NA,CIS CSC v7.1
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,NA,CIS CSC v8.0
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,EDM01.02,COBIT 2019
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,APO01.09,COBIT 2019


[1] 300843      7

In [ ]:
#Remove duplicate PAIRS of regulations and chapter IDs
source_citation_new = source_citation_to_control_objective_new[!duplicated(source_citation_to_control_objective_new[c("chapterID","regulation")]),]

#Alternative method for removing duplicate PAIRS of regulations and chapter IDs 
#distinct(source_citation_to_control_objective, chapterID, regulation, .keep_all= TRUE)

head(source_citation_new, 5) #check the first few rows of new df
dim(source_citation_new) #check the new df dimension

#test removing duplicates on regulation, chapter ID, and control objective name columns
#source_test = source_citation_to_control_objective[!duplicated(source_citation_to_control_objective[c("chapterID","Control Objective Name", "regulation")]),]
#head(source_test, 5) #check the first few rows of new df
#dim(source_test) #check the new df dimension

Control Objective Name,SCF Domain,Control Objective Description,Methods To Comply With SCF Controls,SCF Control Question,chapterID,regulation
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,CC1.2,AICPA TSC 2017 (SOC 2)
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,NA,CIS CSC v7.1
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,NA,CIS CSC v8.0
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,EDM01.02,COBIT 2019
Security & Privacy Governance Program GOV-01,Security & Privacy Governance,Mechanisms exist to facilitate the implementation of cybersecurity and privacy governance controls. - Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP) Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,- Steering committee - Digital Security Program (DSP) - Cybersecurity & Data Protection Program (CDPP),Does the organization staff a function to centrally-govern cybersecurity and privacy controls?,APO01.09,COBIT 2019


[1] 20990     7

#**Exporting cleaned data NEW**

In [ ]:
# Export cleaned scf sheet into new CSV file for Control Objectives table in SN
write.csv(scf_new_cleaned,"/content/scf_new_cleaned.csv", row.names = FALSE)

ERROR: ignored

In [ ]:
# Export regulations into new CSV file for Authority Docs table in SN
write.csv(source_auth_doc_new,"/content/regulations_new.csv", row.names = FALSE)

In [ ]:
# Export citations into new CSV file for Citations table in SN
write.csv(source_citation_new,"/content/citations_new.csv", row.names = FALSE)

In [ ]:
# Export Control Objectives to Citations into new CSV file for relationship table between 
# Control Objectives and Citations in SN
write.csv(source_citation_to_control_objective_new,"/content/citations_to_control_objectives_new.csv", row.names = FALSE)

In [ ]:
# Export transposed data into new CSV file for reference
write.csv(scf_new_cleaned_transposed,"/content/scf_new_cleaned_transposed.csv", row.names = FALSE)

In [ ]:
# Export cleaned authoritative sheet into separate new CSV file
# write.csv(auth_cleaned,"/content/auth_cleaned.csv", row.names = FALSE)

# **Automate Regulatory Change Management**

In [ ]:
dataframe1 = read.csv("/content/regulations.csv")
dataframe2 = read.csv("/content/regulations_new.csv")

dim(dataframe1)

dim(dataframe2)

[1] 234   1

[1] 246   1

In [ ]:
merge(dataframe1, dataframe2, by = 'regulation', type = "inner")

regulation
<chr>
AICPA TSC 2017 (SOC 2)
Americas Argentina
Americas Argentina Reg 132/2018
Americas Bahamas
Americas Bermuda BMA CCC
Americas Brazil LGPD
Americas Canada CSAG
Americas Canada PIPEDA
Americas Chile


In [ ]:
anti_join(dataframe2, dataframe1, by='regulation')

regulation
<chr>
ISO 27002 v2022
NIST 800-171A
NIST 800-218 v1.1
Shared Assessments SIG 2022
US - CO Colorado Privacy Act
US - IL PIPA
US - NY SHIELD Act S5575B
US-TX DIR Control Standards 2.0
US-TX TX-RAMP Level 1
